In [8]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from get_12ECG_features import get_12ECG_features
from driver import load_challenge_data

import sklearn
from sklearn.metrics import mutual_info_score

In [2]:
# Define directories here
data_dir = "/share/workhorse3/vsanil/physionet/Training_WFDB"
data_json_path = "/share/workhorse3/vsanil/physionet/"

In [3]:
# Define program variables here
data_dict = {}

In [4]:
# Extracts features and stores as a dictionary
def extract_all_data(path_name, data_dict):
    # Walk through directory
    for root, dirs, files in os.walk(path_name):
        for filename in files:
            if filename.endswith('.mat'):
                print(os.path.join(root, filename))
                data, header = load_challenge_data(os.path.join(root, filename))
                data_dict[filename.replace('.mat','')] = [data, header]
                
    return data_dict

# Save data in pickle format
def save_pickle(path, data_dict):
    with open(os.path.join(path, 'data.p'), 'wb') as fp:
        pickle.dump(data_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
# Get label dict
def get_labels(data_dict):
    label_dict = {}
    for filename, feature_list in data_dict.items():
        label = feature_list[1][-4].replace("#Dx: ","").replace("\n","").split(',')
        for l in label:
            if l not in label_dict:
                label_dict[l] = 0
        
    return label_dict

In [5]:
# Check if model dictionary is already present
if os.path.exists(os.path.join(data_json_path, 'data.p')):
    with open(os.path.join(data_json_path, 'data.p'), 'rb') as fp:
        data_dict = pickle.load(fp)
else:
    data_dict = extract_all_data(data_dir, data_dict)
    save_pickle(data_json_path, data_dict)
    
# Generate label dictionary
label_dict = get_labels(data_dict)
sex_dict = {"Male":0, "Female":1}

In [27]:
obs_dict = {'Age':[], 'Sex':[], 'Label':[]}
feature_hash = {}
count = 0

for filename, feature_list in data_dict.items():
    feat_lead_12 = feature_list[0]
    header_data = feature_list[1]
    ind_obs_l_dict = {}
    
    # Get metadata 
    label = header_data[-4].replace("#Dx: ","").replace("\n","").split(',')
    age = header_data[-6].replace("#Age: ","").replace("\n","").split(',')[0]
    sex = sex_dict[header_data[-5].replace("#Sex: ","").replace("\n","").split(',')[0]]
    
    if age=="NaN":
        continue
    
    # Generate labels
    for k,v in label_dict.items():
        if k in label:
            ind_obs_l_dict[k] = 1
        else:
            ind_obs_l_dict[k] = 0
    feat_label = list(ind_obs_l_dict.values())
    if tuple(feat_label) not in feature_hash:
        feature_hash[tuple(feat_label)] = count
        count += 1
    
    # Construct a observation dictionary
    obs_dict['Age'].append(int(age))
    obs_dict['Sex'].append(sex)
    obs_dict['Label'].append(feat_label)
    
    
    # Display header
    #for h in header_data:
    #    print(h[:-1])
    
df = pd.DataFrame(obs_dict, columns = ['Age', 'Sex', 'Label'])

In [28]:
df

,Age,Sex,Label
0,28,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,17,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,90,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,63,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,51,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
...,...,...,...
6867,76,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
6868,27,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
6869,82,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0]"
6870,39,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [21]:
df.corr()

,Age,Sex,Label
Age,1.000000,-0.130352,0.085924
Sex,-0.130352,1.000000,-0.051748
Label,0.085924,-0.051748,1.000000


In [14]:
a = np.array([1,2,3,4,5,6,6,7,8,4,2,453,2,13,346,346,23,12,26,31,4351,45,1435,16,7,45848,123])
b = np.array([1,0,0,1,0,1])
c = np.histogram2d(a,b,5)[0]

ValueError: operands could not be broadcast together with shapes (27,) (6,) 

In [25]:
label_dict

{'429622005': 0,
 '164889003': 0,
 '164931005': 0,
 '59118001': 0,
 '284470004': 0,
 '426783006': 0,
 '270492004': 0,
 '164884008': 0,
 '164909002': 0}

In [ ]:
6238 --> [1,0,0,0,0,0,0,1,0]